In [4]:
# MOPTA Competition Base Model 04/13/2024

############ Initialize Tools ############
begin
    import Pkg;
    # Initialize JuMP to allow mathematical programming models
    # Add Packages if you are running this for the first time
    
    
    # Pkg.add("JuMP")
    # Pkg.add("CSV")
    # Pkg.add("DataFrames")
    # Pkg.add("Clp")
    # Pkg.add("PlotlyJS")
    # Pkg.add("Dates")
    # Pkg.add("XLSX")
    # Pkg.add("FileIO")
    # Pkg.add("PrettyTables")
    # Pkg.add("Gurobi")
    # Pkg.add("PyCall")
    
    
    using JuMP
    using CSV
    using DataFrames
    using PlotlyJS
    using Dates
    using XLSX
    using FileIO
    using Base
    using PrettyTables
    using WebIO
    using Gurobi
    # using Clp
    # using Ipopt
end

############ Program Preparations ############
begin
    # Update automatically the date when this program is run.
    today_date = today()
    
    # Please update information of this program to automatically update the code name.
    code_name = "BaseModel"
    version = "1.0"

    folder_name = "$code_name._V$version._$today_date"

    # Create folder to later save data and plots
    begin
        # Define the path for the new folder
        folder_path = "$folder_name"

        # Use mkpath() to create the folder if it doesn't exist
        mkpath(folder_path)
    end

    # Function to save a plot as a PNG file in the specified folder
    function save_plot(plot, path, filename, format="png")
        # Create the full file path with the specified filename and format
        full_path = joinpath(path, string(filename, ".", format))
        
        # Save the plot as an image in the desired format
        savefig(plot, full_path)
    end
end

############ Load Data ############ 
begin
    scenarios = Dict()
    for i in 2001:2020
        file_path = "data_hourly/$(i)_hourly.csv"
        scenarios[i-2000] = CSV.read(file_path, DataFrame)
    end

end

############ Declare Parameters ############
begin
    # Time Horizon Parameters
    begin
        TimeStart = 1;
        # TimeEnd = 365 * 24;
        TimeEnd = 365 * 24;
    end
    # Energy Generation Parameters
    begin
        # Solar PV Parameters
        C_PV = 400 # [$/kW] Capital Cost of Solar PV  
        C_PV_OP = 10 # [$/(kW*YR)] Operational and Maintenance Cost of Solar PV
        η_PVIV = 0.96 # [1] PV(DC) to Home(AC) inverter efficiency

        # Wind Parameters
        C_W = 750 # [$/kW] Capital Cost of Wind Generation 
        C_W_OP = 45 # [$/(kW*YR)] Operational and Maintenance Cost of Wind Generation
    end        
    # Electrolyzer Parameters
    begin
        C_EL = 1000 # [$/kW]
        α_EL = 0.7 # electricity to hydrogen
        k_E2H = (1/0.7)/50 # [kwh electricity to kg hydrogen]
    end
    # Fuel Cell Parameters
    begin
        C_FC = 200 # [$/kW]
        α_FC = 0.75 # hydrogen to electricity
        k_H2E = (1/0.75)*33 # [kg hydrogen to kWh electricity]
    end
    # Storage Parameters
    begin
        L_ss = 0.01 # [/hr] short term leakage rate
        L_ls = 0.03/24 # [/hr] long term leakage rate
        β_l2g = 0.75 # liquid to gas efficiency
        β_g2l = 0.9 # gas to liquid efficiency

        C_l2g = 0 # [$/kg] liquid to gas conversion cost
        C_g2l = 2.75 # [$/kg] gas to liquid conversion cost

        C_ss = 0.33 # [$/kg] gas hydrogen
        C_ls = 1.2 # [$/kg] liquid hydrogen storage
        C_C_ss = 1000 # [$/kg] capacity cost of hydrogen gas storage
        C_C_ls = 1400 # [$/kg] capacity cost of hydrogen liquid storage
    end
    # Transmission & Distribution Parameters
    begin
        μ = 0.995 # transportation efficiency
        C_d = 10 # [$/kg] Distribution cost of hydrogen
    end
    # Economic Parameters
    begin
        Lifetime = 20 # [YR] 
        d = 0.03 # [1] Discount Rate
        CRF = (d*(1+d)^Lifetime)/((1+d)^Lifetime-1) # [1] Capital Recovery Factor
    end
    begin
    # Capacity Parameteres
        UB_WindSize = Inf
        UB_ESize = Inf
    end
end

function Optimize(stepsize)
    ########## Instructions  ##########
    begin
    end
    ########## Data Preparations  ##########  
    begin
        # Set timesteps 
        TIME = collect(TimeStart:1:TimeEnd); # Collect timesteps into a vector
        NumTime = length(TIME); # Number of timesteps, useful for indexing
        δt = stepsize/60 # [hr] Declare stepzize for the optimization program
    end  
    ########## Declare model  ##########
    begin
        # Define the model name and solver. In this case, model name is "m"
        # m = Model(Clp.Optimizer)
        # m = Model(Ipopt.Optimizer)
        # For Gurobi (note that sometimes Clp and Gurobi give slightly different results)
        begin
            # Set path to license (for those using Gurobi)
            ENV["GRB_HOME"] = "C:\\gurobi1101\\win64"
            ENV["GRB_LICENSE_FILE"] = "C:\\Users\\CY\\Documents\\Berkeley\\MOPTA\\gurobi.lic"
            m = Model(Gurobi.Optimizer)
        end
    end
    ######## Decision variables ########

    begin
        # first-stage decision variables
        @variable(m, PVSize >= 0); # [kW] Solar PV Power Capacity
        @variable(m, WindSize >= 0); # [kW] Wind Farm Power Capacity
        @variable(m, ESize >= 0); # [kW] Electrolyzer Power Capacity
        @variable(m, FSize >= 0); # [kW] Hydrogen Fuel Cell Capacity
        @variable(m, SSSize >= 0); # [kg] Short Term Storage Energy Capacity
        @variable(m, LSSize >= 0); # [kg] Long Term Storage Energy Capacity

    end

    begin
        # second stage decision variables
        @variable(m, PV2R[s=1:20, 1:NumTime] >= 0); # [kW] electrical power transfer from PV to residential load

        @variable(m, PV2I[s=1:20, 1:NumTime] >= 0); # [kW] electrical power transfer from PV to industrial load

        @variable(m, W2R[s=1:20, 1:NumTime] >= 0); # [kW] electrical power transfer from wind farm to residential load

        @variable(m, W2I[s=1:20, 1:NumTime] >= 0); # [kW] electrical power transfer from wind to industrial load

        @variable(m, PV2E[s=1:20, 1:NumTime] >= 0); # [kW] electrical power transfer from PV to electrolyzer

        @variable(m, W2E[s=1:20, 1:NumTime] >= 0); # [kW] electrical power transfer from wind to electrolyzer

        @variable(m, E2F[s=1:20, 1:NumTime] >= 0); # [kg] electrolyzer to fuel cell

        @variable(m, E2SS[s=1:20, 1:NumTime] >= 0); # [kg] electrolyzer to short term storage
        
        @variable(m, E2I[s=1:20, 1:NumTime] >= 0); # [kg] electrolyzer to industrial gas load

        @variable(m, F2R[s=1:20, 1:NumTime] >= 0); # [kW] fuel cell to residential load

        @variable(m, F2I[s=1:20, 1:NumTime] >= 0); # [kW] fuel cell to industrial load

        @variable(m, E2LS[s=1:20, 1:NumTime] >= 0); # [kg] electrolyzer to long term storage

        @variable(m, SS2F[s=1:20, 1:NumTime] >= 0); # [kg] hydrogen gas discharged from short term storage to fuel cell

        @variable(m, SS2I[s=1:20, 1:NumTime] >= 0); # [kg] hydrogen gas discharged from short term storage to industrial gas load

        @variable(m, LS2F[s=1:20, 1:NumTime] >= 0); # [kg] hydrogen liquid discharged from long term storage to fuel cell

        @variable(m, LS2I[s=1:20, 1:NumTime] >= 0); # [kg] hydrogen liquid discharged from long term storage to industrial gas load

        @variable(m, InStorageSS[s=1:20, 1:NumTime] >= 0); # [kg] Short term remaining storage

        @variable(m, InStorageLS[s=1:20, 1:NumTime] >= 0); # [kg] Long term remaining storage

        @variable(m, PV2G[s=1:20, 1:NumTime] >= 0); # [kW] electrical power transfer from PV to curtailment

        @variable(m, W2G[s=1:20, 1:NumTime] >= 0); # [kW] electrical power transfer from wind farm to curtailment
    end

    ############ Objective Functions #############
    begin
        # First-stage costs: Capital and fixed operational
        @expression(m, capital_cost, C_PV * PVSize + C_W * WindSize + C_FC * FSize + C_EL * ESize + C_C_ls * LSSize + C_C_ss * SSSize)
        @expression(m, fixed_OM_cost, C_PV_OP * PVSize + C_W_OP * WindSize)
        @expression(m, first_stage_cost, capital_cost * CRF + fixed_OM_cost)

        # Second-stage costs: Variable costs dependent on scenario-specific data
        @expression(m, second_stage_cost[s in keys(scenarios)], 
            sum((InStorageSS[s, t] * C_ss + InStorageLS[s, t] * C_ls + LS2I[s, t] * C_d) for t=1:NumTime) +
            sum((E2LS[s, t] * C_g2l + (LS2F[s, t] + LS2I[s, t]) * C_l2g) for t=1:NumTime))

        # Objective function: Minimize expected total cost
        scenario_probability = 1.0 / length(scenarios)  # Adjust if different probabilities are given
        @objective(m, Min, first_stage_cost + scenario_probability * sum(second_stage_cost[s] for s in keys(scenarios)))

    end
    ############# Expressions ############

    ############# Constraints ############
    begin
        # Loop through each scenario
        for s in keys(scenarios)
            input = scenarios[s]  # Access scenario-specific input data

            # Initialization constraints
            @constraint(m, InStorageSS[s, 1] == 0.5 * SSSize);
            @constraint(m, InStorageSS[s, NumTime] >= 0.5 * SSSize);
            @constraint(m, InStorageLS[s, 1] == 0);

            # Energy balance constraints
            @constraint(m, [t=1:NumTime], input[t, 10] * PVSize * 1000 == PV2E[s, t] + PV2R[s, t] + PV2I[s, t] + PV2G[s, t]);
            @constraint(m, [t=1:NumTime], input[t, 9] * WindSize * 1000 == W2E[s, t] + W2R[s, t] + W2I[s, t] + W2G[s, t]);

            # Energy conversion and efficiency constraints
            @constraint(m, [t=1:NumTime], δt * (W2E[s, t] + PV2E[s, t]) * α_EL * k_E2H == E2F[s, t] + E2SS[s, t] + E2I[s, t] + E2LS[s, t]);
            @constraint(m, [t=1:NumTime], E2F[s, t] + SS2F[s, t] + LS2F[s, t] * β_l2g == δt * (F2R[s, t] + F2I[s, t]) / (α_FC * k_H2E));

            # Storage dynamics and leakage
            @constraint(m, [t=1:NumTime-1], InStorageSS[s, t+1] == (InStorageSS[s, t] + E2SS[s, t] - SS2I[s, t] - SS2F[s, t]) * (1-L_ss));
            @constraint(m, [t=1:NumTime-1], InStorageLS[s, t+1] == (InStorageLS[s, t] + E2LS[s, t] * β_g2l - LS2F[s, t] - LS2I[s, t]) * (1-L_ls));

            # Storage capacity limits
            @constraint(m, [t=1:NumTime], InStorageSS[s, t] <= SSSize);
            @constraint(m, [t=1:NumTime], InStorageLS[s, t] <= LSSize);

            # Demand satisfaction
            @constraint(m, [t=1:NumTime], sum(input[t, i] for i = 4:8) * 1000 == PV2R[s, t] + W2R[s, t] + F2R[s, t]);
            @constraint(m, [t=1:NumTime], sum(input[t, i] for i = 2:3) * 1000 == PV2I[s, t] + W2I[s, t] + F2I[s, t]);
            @constraint(m, [t=1:NumTime], input[t, 11] == E2I[s, t] + SS2I[s, t] + LS2I[s, t] * β_l2g * μ);

            # Capacity constraints
            @constraint(m, [t=1:NumTime], W2E[s, t] + PV2E[s, t] <= ESize);
            @constraint(m, [t=1:NumTime], (F2R[s, t] + F2I[s, t]) <= FSize);
        end
    end
    ########### Solve  ##########
    optimize!(m); 
    ########### Model Results  ##########
    begin
        # Return system sizes and other scalar and time series data

        PV_Size = round(value.(PVSize), digits=2); # [kW]

        Wind_Size = round(value.(WindSize), digits=2); # [kW]

        E_Size = round(value.(ESize), digits=2); # [kW]

        F_Size = round(value.(FSize), digits=2); # [kW]

        SS_Size = round(value.(SSSize), digits=2); # [kg]

        LS_Size = round(value.(LSSize), digits=2); # [kg]
        
        ObjValue = objective_value(m); # [$] Levelized cost of system over lifetime

        # InStorageSS_values = [round(value(InStorageSS[t]), digits=2) for t in TIME]
        # InStorageLS_values = [round(value(InStorageLS[t]), digits=2) for t in TIME]
    end
    results_dict = Dict(
        "Costs" => ObjValue / 10^6, # Converted to millions USD for readability
        "Capacities" => Dict(
            "PV" => PV_Size,
            "Wind" => Wind_Size,
            "Electrolyzer" => E_Size,
            "Fuel Cell" => F_Size,
            "Short Term Storage" => SS_Size,
            "Long Term Storage" => LS_Size
        ),
        # "Flows" => Dict(
        #     "InStorageSS" => InStorageSS_values,
        #     "InStorageLS" => InStorageLS_values,
        #     # Include other variables similarly...
        # )
    )

    return results_dict
end

results = Optimize(60)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 PRO 4650U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2277620 rows, 3504006 columns and 8605859 nonzeros
Model fingerprint: 0xd5bf5ed3
Coefficient statistics:
  Matrix range     [2e-02, 7e+06]
  Objective range  [2e-02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 9e+05]
Presolve removed 251401 rows and 1007179 columns (presolve time = 17s) ...
Presolve removed 524305 rows and 1105005 columns (presolve time = 20s) ...
Presolve removed 881863 rows and 1462563 columns (presolve time = 25s) ...
Presolve removed 881985 rows and 1462564 columns
Presolve time: 32.02s
Presolved: 1395635 rows, 2041442 columns, 6021602 nonzeros

Concurrent LP optimizer: primal simplex, 

Dict{String, Any} with 2 entries:
  "Costs"      => 253.082
  "Capacities" => Dict("Fuel Cell"=>1.06821e6, "Wind"=>445.46, "Electrolyzer"=>…

In [5]:
println(results["Capacities"])

Dict("Fuel Cell" => 1.06820659e6, "Wind" => 445.46, "Electrolyzer" => 1.74335104e6, "Short Term Storage" => 311023.53, "PV" => 203133.48, "Long Term Storage" => 8153.7)
